In [22]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
import dotenv
import os

In [4]:
dotenv.load_dotenv()

api_key = os.getenv('OpenAI_API_Key')

In [5]:
# 마크다운 파일 경로
markdown_paths = ["prompt_engineering_tech.md", "Prompt_Design_and_engineering_introduction_and_advanced_methods.md"]

# 각 마크다운 파일을 로드하고 데이터를 병합
docs = []
for path in markdown_paths:
    loader = UnstructuredMarkdownLoader(path)
    data = loader.load()
    docs.extend(data)

In [6]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [7]:
# 마크다운 헤더를 기준으로 텍스트를 분할하는 MarkdownHeaderTextSplitter 객체를 생성합니다.
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# 분할된 결과를 저장할 리스트
split_data = []

# 각 Document 객체의 텍스트를 분할
for document in docs:
    # Document 객체의 텍스트 추출
    text = document.page_content
    # 텍스트 분할
    md_header_splits = markdown_splitter.split_text(text)
    # 분할된 결과를 저장
    split_data.extend(md_header_splits)

In [8]:
chunk_size = 250
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
splits

[Document(page_content='1. INTRODUCTION  \n1.1 What is Prompt?  \n: Prompt는 생성형 AI 모델에서 모델의 출력 결과를 가이드하기 위해서 사용자가 입력하는 문자적 입력. 이것은 단순한 질문에서 자세한 묘사나 특정 업무일수도 있음.  \nDALLE-3 같은 Context-Image Generation model에서는, Prompt는 종종 묘사적 성격을 띔.'),
 Document(page_content='반면에 GPT-4, Gemini 같은 LLM에서는 간단한 쿼리부터, 복잡한 문제 설명문까지 광범위함.  \nPrompt 의 구성 요소\n(AI모델에서 원하는 답변을 얻기 위해서는 지침과 질문이 필수적이며, 그 외 요소들은 선택적임)\ninstruction(지침)\nquestions(질문)\ninputs data(입력 값)\nexamples(예시)  \n기본적인 Prompt는 직접적인 질문이나 특정 일에 대한 지침을 제공하는 것처럼 단순함'),
 Document(page_content='발전된 형태의 Prompt는 보다 복잡한 형태의 구조로 구성되어 있는데, CoT prompting 같이 정답에 도달하게 하기 위해서 논리적 추론 구조를 가이드 하게하는 방식을 따름.  \n1.2 Basic prompt examples  \n1.1 에서 언급했듯 Prompt는 지침과 질문, 입력값 그리고 예시로 설계된다.  \n결과를 출력하기 위해서 반드시 필요한 것: 지침, 질문  \n그 외: 옵션 개념이라 이해하면 됨(GPT-4 이용)'),
 Document(page_content='그 외: 옵션 개념이라 이해하면 됨(GPT-4 이용)  \n1.2.1 Instructions + Question  \n단순한 질문을 하는 것 외에, Prompt 의 다음 단계는 몇 지침들을 포함하는 것  \n각 지침들은 모델이 어떻게 질문에 답변해야하는지를 기록했음.  \n예시: 대학교 에세이 작성 방법에 대해서 조언 구하기\nmarkdown'

In [27]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = '''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:'''

prompt_temp = PromptTemplate.from_template(prompt)

In [24]:
# 생성 모델 초기화
llm = OpenAI(api_key=api_key)


In [28]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_temp
    | llm
    | StrOutputParser()
)

In [29]:
rag_chain.invoke("What is Chain of Thought?")

' Chain of Thought is a technique used to promote a more logical and accurate response from a model by forcing it to follow a series of steps during the inference process. This method helps the model to not only come to a conclusion, but also go through a logical reasoning process to reach that conclusion. It can also help improve the factual accuracy of the model by prompting it to cite correct sources.'